## Решение СЛАУ методом QR-разложения.

In [37]:
from numpy.linalg import solve, cond, norm
from prettytable import PrettyTable
from numpy import array, column_stack, matrix, ones, zeros, dot

Метод вращений

In [27]:
def rotation_method(A, b):
    ab = column_stack([A, b])
    n = ab.shape[0]
    for i in range(n-1):
        for j in range(i + 1, n):
            c = ab[i, i] / (ab[i, i]**2 + ab[j, i]**2) ** 0.5
            s = ab[j, i] / (ab[i, i]**2 + ab[j, i]**2) ** 0.5
            temp1 = ab[i, :] * c + ab[j, :] * s
            temp2 = ab[i, :] * -s + ab[j, :] * c
            ab[i, :] = temp1
            ab[j, :] = temp2

    x = matrix([0.0 for i in range(n)]).transpose()
    for i in range(n - 1, -1, -1):
        x[i, 0] = (ab[i, -1] - ab[i, i:n] * x[i:n, 0]) / ab[i, i]

    return x

Метод квадратного корня

In [39]:
class SquareRootMethod:
    def __init__(self, A, b):
        self.n = A.shape[0]
        self.A = A
        self.b = b
        self.L = zeros((self.n, self.n))

    def __sum_one(self, i_first):
        s = 0
        for i in range(i_first):
            s += self.L[i_first][i]**2
        return s

    def __L_ii(self, i):
        self.L[i][i] = sqrt(self.A[i][i] - self.__sum_one(i))

    def __sum_two(self, i_first, j_first):
        s = 0
        for i in range(i_first):
            s += self.L[i_first][i]*self.L[j_first][i]
        return s

    def __L_ij(self, i_first, n):
        for i in range(i_first+1, self.n):
            self.L[i][i_first] = (self.A[i][i_first] - self.__sum_two(i_first, i))/self.L[i_first][i_first]

    def matrix_L(self):
        for i in range(self.n):
            self.__L_ii(i)
            self.__L_ij(i, self.n)
        return self.L
    
    def solve_x(self):
        self.matrix_L()
        y = solve(self.L, self.b)
        x = solve(self.L.T, y)
        return x

In [40]:
def hilbert_matrix(n: int):
    matrix = zeros((n, n))
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            matrix[i - 1][j - 1] = 1 / (i + j - 1)
    return matrix

In [45]:
for size in [3, 6, 12]:
    x = ones(size)
    A = hilbert_matrix(size)
    b = dot(A, x)
    print('Матрица Гильберта', size, 'x', size)
    print('Число обусловленности =', cond(A))
    print('Метод вращений = ', norm(x - rotation_method(A, b)))
    print('Метод квадратного корня = ', norm(x - SquareRootMethod(A, b).solve_x()))
    print()

Матрица Гильберта 3 x 3
Число обусловленности = 524.0567775860644
Метод вращений =  2.8398063987129677e-14
Метод квадратного корня =  6.157485465446498e-15

Матрица Гильберта 6 x 6
Число обусловленности = 14951058.642466014
Метод вращений =  3.482286278236852e-09
Метод квадратного корня =  5.998650300168924e-10

Матрица Гильберта 12 x 12
Число обусловленности = 1.7587184385400606e+16
Метод вращений =  1.5499142010568772
Метод квадратного корня =  0.8972942714050014

